In [6]:
# Import dependencies
import pandas as pd
from sodapy import Socrata

In [9]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("chronicdata.cdc.gov", None)

In [10]:
# First 2000 results, returned as csv from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("swc5-untb", limit=2000)

In [11]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [13]:
# Display dataframe
results_df.head()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,...,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation,:@computed_region_bxsw_vy29,:@computed_region_he4y_prf8
0,2020,WY,Wyoming,Johnson,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Age-adjusted prevalence,9.9,...,10.4,8588,56019,HLTHOUT,CASTHMA,AgeAdjPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-106.5885406...",14,3118
1,2020,WY,Wyoming,Albany,BRFSS,Health Outcomes,Diagnosed diabetes among adults aged >=18 years,%,Age-adjusted prevalence,7.8,...,8.4,38950,56001,HLTHOUT,DIABETES,AgeAdjPrv,Diabetes,"{'type': 'Point', 'coordinates': [-105.7218826...",14,3079
2,2020,WY,Wyoming,Hot Springs,BRFSS,Health Outcomes,Chronic kidney disease among adults aged >=18 ...,%,Crude prevalence,3.9,...,4.3,4425,56017,HLTHOUT,KIDNEY,CrdPrv,Chronic Kidney Disease,"{'type': 'Point', 'coordinates': [-108.4350765...",14,3117
3,2020,WY,Wyoming,Sheridan,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,19.9,...,20.9,30863,56033,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis,"{'type': 'Point', 'coordinates': [-106.8812114...",14,3123
4,2020,WY,Wyoming,Sweetwater,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Age-adjusted prevalence,6.1,...,6.9,42673,56037,HLTHOUT,COPD,AgeAdjPrv,COPD,"{'type': 'Point', 'coordinates': [-108.8756774...",14,3125
